In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
buildingo = pd.read_csv('Building_Ownership_Use.csv')
buildingc = pd.read_csv('Building_Structure.csv')
test = pd.read_csv('test.csv')

In [3]:
combin = pd.concat([train, test])
del(train,test)

/home/tinku/.virtualenvs/cv/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [4]:
combin = pd.merge(left = combin, right = buildingo, on = ['building_id','vdcmun_id', 'district_id'] )

In [5]:
combin = pd.merge(left = combin, right = buildingc, on = ['building_id','vdcmun_id', 'district_id', 'ward_id'] )

In [6]:
del( buildingo, buildingc)

In [7]:
combin_na = (combin.isnull().sum()/len(combin))
combin_na = combin_na.drop(combin_na[combin_na==0].index).sort_values(ascending = False)*100
missing_data = pd.DataFrame({'Missing': combin_na})
missing_data

,Missing
damage_grade,40.000057
has_repair_started,5.255685
count_families,0.000095


In [8]:
combin.count_families.fillna(1, inplace = True)
combin.has_repair_started.fillna(0, inplace = True)

In [9]:
del(missing_data, combin_na)

In [11]:
combin.area_assesed.unique()
combin['area_assesed_count'] = combin.groupby('area_assesed')['area_assesed'].transform('count')

In [12]:
combin.drop('building_id', axis = 1, inplace = True)

In [13]:
combin['district_id_count'] = combin.groupby('district_id')['district_id'].transform('count')

In [14]:
combin['vdcmun_id_count'] = combin.groupby('vdcmun_id')['vdcmun_id'].transform('count')

In [15]:
combin['ward_id_count'] = combin.groupby('ward_id')['ward_id'].transform('count')

In [16]:
combin['district_vdcmun_count'] = combin.groupby(['district_id', 'vdcmun_id'])['district_id'].transform('count')

In [17]:
combin['district_ward_vdc_count'] = combin.groupby(['district_id', 'vdcmun_id', 'ward_id'])['district_id'].transform('count')

In [18]:
combin['floor_gap'] = ( combin['count_floors_pre_eq'] - combin['count_floors_post_eq'])
combin['height_gap'] = ( combin['height_ft_pre_eq'] - combin['height_ft_post_eq'])

In [19]:
combin['land_surface_condition_district'] = combin.groupby(['land_surface_condition', 'district_id'])['district_id'].transform('count')
combin['condition_post_district'] = combin.groupby(['condition_post_eq', 'district_id'])['district_id'].transform('count')

In [20]:
combin['condition_post_district_vdc_ward'] = combin.groupby(['condition_post_eq', 'district_id', 'vdcmun_id', 'ward_id'])['district_id'].transform('count')

In [21]:
combin['area_district'] = combin.groupby(['area_assesed', 'district_id'])['district_id'].transform('count')

In [22]:
combin['area_district_vdc_ward'] = combin.groupby(['area_assesed', 'district_id', 'vdcmun_id', 'ward_id'])['district_id'].transform('count')

In [23]:
#combin = combin.drop(['district_id'], axis = 1)
combin['district_id'] = combin['district_id'].astype('category')

In [24]:
combin['plinth_area_sq_ft'] = np.log1p(combin['plinth_area_sq_ft'])

In [25]:
combin['plint_avg_district'] = combin.groupby(['district_id'])['plinth_area_sq_ft'].transform(lambda x: x.sum() / x.count())

In [26]:
combin['plint_avg_district_vdc'] = combin.groupby(['district_id','vdcmun_id'])['plinth_area_sq_ft'].transform(lambda x: x.sum() / x.count())

In [27]:
combin = combin.drop(['vdcmun_id'], axis = 1)

In [28]:
combin = combin.drop(['ward_id'], axis = 1)

In [29]:
train2 = combin[combin.damage_grade.isnull() == False]
test2 = combin[combin.damage_grade.isnull() == True]
del(combin)

In [31]:
test2 = test2.drop(['damage_grade'], axis = 1)

In [32]:
X = train2.drop(['damage_grade'], axis = 1)
X = pd.get_dummies(X, drop_first=True)
test2 = pd.get_dummies(test2, drop_first=True)
y = train2['damage_grade']
del(train2)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)

In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

In [35]:
import lightgbm as lgb
dtrainlgb = lgb.Dataset(data = X_train, label = y_train)
params = {'objective': 'softmax',
         'learning_rate' : 0.2,
          'num_class' : 5,
          'num_leaves' : 60}
cv_results = lgb.cv(params,
                    dtrainlgb,
                    num_boost_round = 1000,
                    nfold = 3,
                    metrics = 'multi_error',
                    early_stopping_rounds = 20,
                    verbose_eval = 5)           #600 iterations

[5]	cv_agg's multi_error: 0.269535 + 0.000965866
[10]	cv_agg's multi_error: 0.264465 + 0.00127746
[15]	cv_agg's multi_error: 0.259414 + 0.0011189
[20]	cv_agg's multi_error: 0.255432 + 0.00105945
[25]	cv_agg's multi_error: 0.251847 + 0.000843365
[30]	cv_agg's multi_error: 0.24877 + 0.000962879
[35]	cv_agg's multi_error: 0.245801 + 0.00114567
[40]	cv_agg's multi_error: 0.243757 + 0.000872258
[45]	cv_agg's multi_error: 0.241882 + 0.00111999
[50]	cv_agg's multi_error: 0.240363 + 0.00109845
[55]	cv_agg's multi_error: 0.238918 + 0.000900385
[60]	cv_agg's multi_error: 0.237848 + 0.000870465
[65]	cv_agg's multi_error: 0.236826 + 0.00076757
[70]	cv_agg's multi_error: 0.236168 + 0.000596701
[75]	cv_agg's multi_error: 0.235589 + 0.00067433
[80]	cv_agg's multi_error: 0.234831 + 0.000904772
[85]	cv_agg's multi_error: 0.234176 + 0.000771494
[90]	cv_agg's multi_error: 0.233524 + 0.000789125
[95]	cv_agg's multi_error: 0.233115 + 0.000722386
[100]	cv_agg's multi_error: 0.232595 + 0.000911347
[105]	cv_a

In [36]:
lgb_model = lgb.train(params,
                     dtrainlgb,
                     num_boost_round = 400,
                     verbose_eval = True)

In [37]:
import matplotlib.pyplot as plt
a = lgb.plot_importance(lgb_model, max_num_features=20, height=.2, figsize=(12,9))
plt.show()

<Figure size 1200x900 with 1 Axes>

In [38]:
lgb_pred = lgb_model.predict(X_test)

In [39]:
dataf = pd.DataFrame(lgb_pred)

In [40]:
dataf.head()

,0,1,2,3,4
0,3.701786e-09,1.690384e-09,8.728171e-08,0.000001,9.999989e-01
1,2.166574e-02,1.412061e-01,4.397540e-01,0.397047,3.268339e-04
2,5.715507e-03,1.048207e-01,4.565497e-01,0.426296,6.618346e-03
3,1.457431e-09,4.091758e-09,2.704027e-07,0.000001,9.999986e-01
4,9.999847e-01,1.143300e-05,1.077746e-06,0.000003,3.078848e-09


In [41]:
dataf['max_col'] = dataf.idxmax(axis=1)

In [42]:
dataf.head()

,0,1,2,3,4,max_col
0,3.701786e-09,1.690384e-09,8.728171e-08,0.000001,9.999989e-01,4
1,2.166574e-02,1.412061e-01,4.397540e-01,0.397047,3.268339e-04,2
2,5.715507e-03,1.048207e-01,4.565497e-01,0.426296,6.618346e-03,2
3,1.457431e-09,4.091758e-09,2.704027e-07,0.000001,9.999986e-01,4
4,9.999847e-01,1.143300e-05,1.077746e-06,0.000003,3.078848e-09,0


In [43]:
dataf['class'] = 'Grade 1'
dataf.loc[dataf['max_col'] == 1, 'class'] = 'Grade 2'
dataf.loc[dataf['max_col'] == 2, 'class'] = 'Grade 3'
dataf.loc[dataf['max_col'] == 3, 'class'] = 'Grade 4'
dataf.loc[dataf['max_col'] == 4, 'class'] = 'Grade 5'

In [44]:
from sklearn.metrics import f1_score
accu = f1_score(y_test, dataf['class'], average='weighted')

In [45]:
accu

0.7834203557674116

In [46]:
submit = lgb_model.predict(test2)


In [47]:
submit2 = pd.DataFrame(submit)

In [48]:
submit2.head()
submit2['max_col'] = submit2.idxmax(axis=1)

In [49]:
submit2['class'] = 'Grade 1'
submit2.loc[submit2['max_col'] == 1, 'class'] = 'Grade 2'
submit2.loc[submit2['max_col'] == 2, 'class'] = 'Grade 3'
submit2.loc[submit2['max_col'] == 3, 'class'] = 'Grade 4'
submit2.loc[submit2['max_col'] == 4, 'class'] = 'Grade 5'

In [51]:
test = pd.read_csv('test.csv')
output = pd.DataFrame({'building_id': test.building_id, 'damage_grade' : submit2['class']})
output.to_csv('output.csv', index=False)